In [ ]:
import sys
import os
import pandas as pd

os.chdir("..")
os.chdir(r"C:\Users\muhammad\Desktop\ir")

from services.evaluation.bert_evaluator import evaluate_bert_model
from services.retrieval.bert_retrieval import BertSearcher
from services.document_store.config import DATASETS

results_list = []
datasets_to_eval = ["antique", "quora"]

for dataset_key in datasets_to_eval:
    dataset_name = DATASETS[dataset_key]
    searcher = BertSearcher(dataset_key)
    results = evaluate_bert_model(searcher, dataset_name ,dataset_key)
    row = {"Model": f"BERT ({dataset_key})"}
    row.update(results)
    results_list.append(row)

df = pd.DataFrame(results_list).set_index("Model")
display(df)


Loading BERT search artifacts for 'antique'...


c:\Users\muhammad\Desktop\ir\venv310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ BERT Searcher initialized successfully.
Loading qrels for antique/test...
Evaluating BERT model...


100%|██████████| 200/200 [05:03<00:00,  1.52s/it]


✅ Saved JSON to C:\Users\muhammad\Desktop\ir\data\evaluation_results\antique_bert_evaluation_results.json
Loading BERT search artifacts for 'quora'...
✅ BERT Searcher initialized successfully.
Loading qrels for beir/quora/test...
Evaluating BERT model...


100%|██████████| 10000/10000 [5:41:41<00:00,  2.05s/it] 

✅ Saved JSON to C:\Users\muhammad\Desktop\ir\data\evaluation_results\quora_bert_evaluation_results.json


,Precision@10,Recall,MRR,MAP
Model,,,,
BERT (antique),0.46300,1.0,0.849043,0.231343
BERT (quora),0.13284,1.0,0.864390,0.837902


In [2]:
import sys
import os
import pandas as pd

os.chdir("..")
os.chdir(r"C:\Users\muhammad\Desktop\ir")

from services.evaluation.bert_evaluator import evaluate_bert_model
from services.retrieval.bert_retrieval import BertSearcher
from services.document_store.config import DATASETS

results_list = []
datasets_to_eval = ["antique", "quora"]

for dataset_key in datasets_to_eval:
    dataset_name = DATASETS[dataset_key]
    searcher = BertSearcher(dataset_key ,use_faiss = True)
    results = evaluate_bert_model(searcher, dataset_name ,dataset_key,use_faiss = True)
    row = {"Model": f"BERT ({dataset_key})"}
    row.update(results)
    results_list.append(row)

df = pd.DataFrame(results_list).set_index("Model")
display(df)


Loading BERT artifacts for 'antique' (Faiss enabled: True)...
✅ Faiss index loaded.
✅ BERT Searcher initialized successfully.
Loading qrels for antique/test...
Evaluating BERT model (using Faiss: True)...


100%|██████████| 200/200 [03:19<00:00,  1.00it/s]


✅ Saved JSON to C:\Users\muhammad\Desktop\ir\data\evaluation_results\antique_bert_faiss.json
Loading BERT artifacts for 'quora' (Faiss enabled: True)...
✅ Faiss index loaded.
✅ BERT Searcher initialized successfully.
Loading qrels for beir/quora/test...
Evaluating BERT model (using Faiss: True)...


100%|██████████| 10000/10000 [4:06:16<00:00,  1.48s/it] 


✅ Saved JSON to C:\Users\muhammad\Desktop\ir\data\evaluation_results\quora_bert_faiss.json


,Precision@10,Recall,MRR,MAP
Model,,,,
BERT (antique),0.46300,1.0,0.849043,0.231341
BERT (quora),0.13285,1.0,0.864542,0.838156


In [1]:
import sys
import os
import pandas as pd
from tqdm import tqdm

# إعداد المسار
os.chdir("..")
os.chdir(r"C:\Users\muhammad\Desktop\ir")

from services.retrieval.bert_retrieval import BertSearcher
from services.rag.generator import TextGenerator
from services.document_store.config import DATASETS
import ir_datasets
from transformers import logging as hf_logging
hf_logging.set_verbosity_error()

# إعداد الباحث والمولد
datasets_to_eval = ["antique"]
results_list = []

for dataset_key in datasets_to_eval:
    dataset_name = DATASETS[dataset_key]
    searcher = BertSearcher(dataset_key, use_faiss=True)
    generator = TextGenerator()
    
    dataset = ir_datasets.load(dataset_name)
    queries = list(dataset.queries_iter())
    
    print(f"Running RAG evaluation for {dataset_key} ...")
    rag_rows = []
    for q in tqdm(queries[:100]):   # خذ أول 100 كويري فقط كمثال، غير الرقم حسب الحاجة
        # 1. استرجاع المستندات الأكثر صلة
        retrieved = searcher.search(q.text, top_k=10)
        doc_ids = [doc_id for doc_id, score in retrieved]
        
        # 2. تحميل نصوص المستندات (مباشرة من ir_datasets أو من db حسب نظامك)
        docs_texts = []
        for doc_id in doc_ids:
            try:
                doc = dataset.docs_store().get(doc_id)
                docs_texts.append(doc.text if doc else "")
            except:
                docs_texts.append("")
        
        context = " ".join(docs_texts)
        
        # 3. توليد إجابة عبر RAG
        generated_answer = generator.answer_from_context(context, q.text)
        
        rag_rows.append({
            "query_id": q.query_id,
            "query_text": q.text,
            "generated_answer": generated_answer,
            "top_docs": docs_texts
        })
    
    # حفظ النتائج
    df = pd.DataFrame(rag_rows)
    out_path = f"rag_eval_{dataset_key}.csv"
    df.to_csv(out_path, index=False)
    print(f"Results saved to {out_path}")
    df = pd.DataFrame(rag_rows)
    display(df.head())



c:\Users\muhammad\Desktop\ir\venv310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading BERT artifacts for 'antique' (Faiss enabled: True, RAG enabled: False)...
✅ Faiss index loaded.
✅ BERT Searcher initialized successfully.
Loading text generation model: t5-small...
✅ Generation model loaded.
Running RAG evaluation for antique ...


100%|██████████| 100/100 [03:15<00:00,  1.96s/it]

Results saved to rag_eval_antique.csv


,query_id,query_text,generated_answer,top_docs
0,3990512,how can we get concentration onsomething?,Try Yoga for concentration,"[In order to have mental peace,and to improve ..."
1,714612,Why doesn't the water fall off earth if it's ...,Answer:,[This goes along with the question of why don'...
2,2528767,How do I determine the charge of the iron ion ...,Question: How do I determine the charge of...,[charge of Fe in Fecl3 is 3. . iron has either...
3,821387,I have mice.How do I get rid of them humanely?,If you are seriously looking for a humane way ...,[Get rid of the mice or whatever it is they ea...
4,1880028,"What does ""see Leaflet"" mean on Ept Pregnancy ...","Question: What does ""see Leaflet"" mean on Ept ...",[Leaflet is the info inside the package with t...
